# Create paper list from .bib file.

## 1. Build bibtexparser

In [1]:
# coding = utf-8
import bibtexparser as bp
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import convert_to_unicode

In [2]:
bibfile_name = "paperlist.bib"

In [3]:
with open(bibfile_name) as bibfile:
    parser = BibTexParser()
    parser.customization = convert_to_unicode
    bibdata = bp.load(bibfile, parser=parser)

## 2. Parse .bib file

In [4]:
# sort by year
biblist = bibdata.entries
biblist.sort(key=lambda x: x['year'], reverse=True)

In [5]:
# other keywords, e.g. paper or code links
other_kws = {"paper_link": "paper", "code_link": "code"}

In [6]:
bibmd = []
for i in range(len(biblist)):
    author = biblist[i]['author'].replace(",", "")
    author = author.replace(" and", ",")
    year = biblist[i]['year']
    title = biblist[i]['title']
    bibmd_string = f"1. **{title}** *{author}*. {year}."
    for kw in other_kws:
        if kw in biblist[i]:
            bibmd_string += f" [{other_kws[kw]}]({biblist[i][kw]})"
    bibmd.append(bibmd_string)
    bibmd.append("\n")

## 3. Create .md file

In [7]:
from mdutils import Html
from mdutils.mdutils import MdUtils

In [8]:
file_name = "paperlist.md"
title = "Paper List of Adversarial Robustness in Graph Machine Learning"
mdFile = MdUtils(file_name, title)

In [9]:
mdFile.new_paragraph("This list collects papers related to the adversarial robustness in graph machine learning.")
mdFile.new_list(bibmd, marked_with=' ')
mdFile.create_md_file()